In [7]:
import typing as tp
import pandas as pd
import numpy as np
import argparse
import os

from wbia_miew_id.helpers import get_config
from scripts.custom_utils import load_miewid_data
from pathlib import Path
from tqdm import tqdm

from scripts.GradCamPlusPlusVisualizer import GradCamPlusPlusVisualizer
from scripts.LightGlueVisualizer import LightGlueVisualizer
from scripts.visualizer import Visualizer

In [16]:
def pipeline(
        visualizer: Visualizer,
        images: tp.List[np.ndarray],
        topk_idx: np.ndarray,
        match_mat: np.ndarray,
        dist_mat: np.ndarray,
        stop_after: int
        ):
    # Iterate through instance-match-indicator tuples
    idx_to_match_idxs = {
        idx:{
            "topk_indices":     topk_idx[idx].tolist(),
            "match_indicators": match_mat[idx].tolist()
            }
        for idx in range(len(images))
    }
    results = []
    for i, (query_idx, vals) in enumerate(tqdm(idx_to_match_idxs.items())):
        if i >= stop_after:
            break
        
        image_query = images[query_idx]

        # Extract top-k indices of matches and whether they were true matches
        topk_indices = vals["topk_indices"]
        match_indicators = vals["match_indicators"]

        # Iterate through top-k matches
        for rank, (match_idx, is_match) in enumerate(tqdm(zip(topk_indices, match_indicators))):
            image_match = images[match_idx]
            
            #print(f"Vizualizing query {query_idx} and match {match_idx}")

            # Generate results from the visualization method
            result = visualizer.generate(
                image_query,
                image_match,
                is_match=is_match,
                topk_indices=topk_indices,
                query_idx=query_idx,
                match_idx=match_idx
            )

            score = dist_mat[query_idx, match_idx]

            # Store computed results
            data = {
                "query_idx":    query_idx,
                "match_idx":    match_idx,
                "score":        score,
                "rank":         rank,
                "is_match":     is_match,
                **result
            }
            
            results.append(data)
            
    return results


In [17]:
### Path setup
config_path = "wbia_miew_id/examples/beluga_example_miewid/benchmark_model/miew_id.msv2_all.yaml"
config = get_config(config_path)
savepath="/srv/transparency/wildbook_prototype/data/matches/figures/"
os.makedirs(savepath, exist_ok=True)
root = Path("wbia_miew_id/examples")
visualization_output_dir = Path(root, f"{config.checkpoint_dir}/{config.project_name}/{config.exp_name}/visualizations")
os.makedirs(visualization_output_dir, exist_ok=True)

### Load precomputed matches generated by MiewID
df_test, test_dataset, match_results, q_pids, topk_idx, topk_names, match_mat, dist_mat, images = load_miewid_data(visualization_output_dir)

### Set up dataframe to store computed results
df_all = pd.DataFrame(columns=[])

### Generate visualizations
lightglue_results = pipeline(
                        LightGlueVisualizer(),
                        images,
                        topk_idx,
                        match_mat,
                        dist_mat,
                        stop_after=1
)
gradcam_results = pipeline(
                        GradCamPlusPlusVisualizer(
                            root,
                            visualization_output_dir,
                            df_test,
                            test_dataset,
                            match_mat
                        ),
                        images,
                        topk_idx,
                        match_mat,
                        dist_mat,
                        stop_after=1
)

Loading config from path: wbia_miew_id/examples/beluga_example_miewid/benchmark_model/miew_id.msv2_all.yaml


  0%|          | 0/849 [00:00<?, ?it/s]

100%|██████████| 849/849 [00:06<00:00, 129.90it/s]


Loaded images: torch.Size([849, 3, 440, 440])


20it [00:03,  6.15it/s][00:00<?, ?it/s]
  0%|          | 1/849 [00:03<45:59,  3.25s/it]


...


/home/cabidi/XAI/wbia-plugin-miew-id/wbia_miew_id/datasets/default_dataset.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {"image": image, "label":torch.tensor(row['name']),
100%|██████████| 2/2 [00:00<00:00, 35.20it/s]


generating embeddings



/home/cabidi/XAI/wbia-plugin-miew-id/wbia_miew_id/datasets/default_dataset.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {"image": image, "label":torch.tensor(row['name']),
100%|██████████| 2/2 [00:00<00:00, 33.73it/s]
/home/cabidi/XAI/wbia-plugin-miew-id/benchmarking/BN_Inception.py:260: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight)
/home/cabidi/XAI/wbia-plugin-miew-id/benchmarking/BN_Inception.py:262: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.weight, 1)
/home/cabidi/XAI/wbia-plugin-miew-id/benchmarking/BN_Inception.py:263: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.bias, 0)
/home/cabidi/XAI/wbia-plugin-miew-id/benchmarking/BN_In

load model from Multi_similarity.pth.tar


FileNotFoundError: [Errno 2] No such file or directory: 'Multi_similarity.pth.tar'